In [2]:
import pandas as pd
import json

In [3]:
# Topic/Employee Admin Parameters/sheet columns
adminList = ["TopicGoals", "EmpRoster", "EmpAffinity"]

adminCols = {"TopicGoals": ["Topic","Goal","std_hours"], 
             "EmpRoster": ["EmpNum","Last","First","Title","Department"], 
             "EmpAffinity": ["EmpNum","Last","First","Title","Topic","SkillRating_1to5"]}

# Timesheet parameters/sheet columns
TopicList = ["CAPA",
"Inspection",
"New Product",
"Audit",
"Training"
]

TopicCols = {
    "CAPA": ["IssueNum","TaskType","EmpNum","Hours","Date","Status","Product"],
"Inspection": ["InspectionNum","Product","InspType","EmpNum","Hours","Date","Status"],
"New Product": ["ProjectNum", "Product","TaskType","EmpNum","Hours","Date","Status"],
"Audit": ["AuditNo","Lead","Duration","TargetDate","ActualDate","Type","Title"],
"Training": ["Instructor","Duration","TargetDate","ActualDate","Type","DocNo","DocName","RefNum"]
}

sheets = adminList + TopicList
colLookups = {**adminCols, **TopicCols}

# desiredCols = [Topic, RefNum, Detail, SubType, Date, Hours, Status, EmpNum, EmpName]
renameDict ={
        "CAPA": {"IssueNum":"RefNum",
                 "Product": "Detail",
                 "TaskType": "SubType",
                 "EmpNum": "EmpNum",
                 "Hours": "Hours",
                 "Date": "Date",
                 "Status": "Status"},
"Inspection": {"InspectionNum": "RefNum",
               "Product": "Detail",
               "InspType": "SubType",
               "EmpNum": "EmpNum",
               "Hours": "Hours",
               "Date": "Date",
               "Status": "Status"},
"New Product": {"ProjectNum":"RefNum",
                "Product": "Detail",
                "TaskType": "SubType",
                "EmpNum": "EmpNum",
                "Hours": "Hours",
                "Date": "Date",
                "Status": "Status"},
"Audit": {"AuditNo": "RefNum",
          "Lead": "EmpNum",
          "Duration": "Hours",
#           "TargetDate",
          "ActualDate": "Date",
          "Type": "SubType",
         "Title": "Detail"},
"Training": {"Instructor": "EmpNum",
             "Duration": "Hours",
#              "TargetDate",
             "ActualDate": "Date",
             "Type": "SubType",
             "DocNo": "Detail",
#              "DocName"
             "RefNum": "RefNum"
            }
}

In [4]:
xlsx = pd.ExcelFile('TimeSheetGenerator.xlsx')

In [5]:
# QUALITY CHECK: Are expected sheets and columns present?
sheetCheck = {}
for sheet in sheets:
    if sheet in xlsx.sheet_names:
        sheetCheck[sheet] = sheet + " is present."
    else:
        sheetCheck[sheet] = "Error: "+ sheet + " NOT is present."

In [6]:
sheetCheck

{'TopicGoals': 'TopicGoals is present.',
 'EmpRoster': 'EmpRoster is present.',
 'EmpAffinity': 'EmpAffinity is present.',
 'CAPA': 'CAPA is present.',
 'Inspection': 'Inspection is present.',
 'New Product': 'New Product is present.',
 'Audit': 'Audit is present.',
 'Training': 'Training is present.'}

In [7]:
colCheck = {}
for sheet in sheets:
    checksheet = xlsx.parse(sheet_name=sheet)
    for col in colLookups[sheet]:
        if col not in checksheet.columns:
            colCheck[sheet] = "Error: Column "+ col + " is NOT present."
        else:
            colCheck[sheet] = "All columns present."

In [8]:
colCheck

{'TopicGoals': 'All columns present.',
 'EmpRoster': 'All columns present.',
 'EmpAffinity': 'All columns present.',
 'CAPA': 'All columns present.',
 'Inspection': 'All columns present.',
 'New Product': 'All columns present.',
 'Audit': 'All columns present.',
 'Training': 'All columns present.'}

In [9]:
# Create dictionary of dataframes with only needed columns from colLookups
dfDict = {}
for sheet in sheets:
    dfDict[sheet]=pd.read_excel(xlsx,sheet)[colLookups[sheet]].dropna()
    

In [10]:
# Add Status to Audit and Training for On-time/Late
for topic in ["Audit","Training"]:
    StatusList = []
    for index, row in dfDict[topic].iterrows():
        if row["ActualDate"] > row["TargetDate"]:
            StatusList.append("Late")
        else:
            StatusList.append("On Time")
    dfDict[topic]["Status"] = StatusList

In [11]:
# Standardize column names
for topic in TopicList:
#     print(topic)
    dfDict[topic]=dfDict[topic].rename(columns = renameDict[topic])
    dfDict[topic]["Topic"] = topic
    dfDict[topic] = dfDict[topic][["Topic", "RefNum", "Detail", "SubType", 
                                  "Date", "Hours", "Status", "EmpNum"]].astype({"EmpNum":"int"})
    dfDict[topic] = dfDict[topic].astype({"EmpNum": "str"})

In [12]:
# combine all dfs
fulldf = pd.concat([dfDict[topic] for topic in TopicList])
# fulldf

In [13]:
# set up lookups for name and title
dfDict["EmpRoster"]["EmpName"] = dfDict["EmpRoster"]["First"].str[0]+" "+dfDict["EmpRoster"]["Last"]
infodict = dfDict["EmpRoster"][["EmpNum","EmpName","Title"]].astype({"EmpNum":"str"}).set_index("EmpNum").to_dict(orient="index")
infodict

{'32301': {'EmpName': 'J Scott', 'Title': 'Quality Supervisor'},
 '49570': {'EmpName': 'Z Taylor', 'Title': 'Inspector'},
 '77030': {'EmpName': 'B Cranston', 'Title': 'Quality Analyst'},
 '77707': {'EmpName': 'T Kwan', 'Title': 'Quality Coordinator'},
 '29436': {'EmpName': 'K Hart', 'Title': 'Inspector'},
 '88927': {'EmpName': 'T Oliver', 'Title': 'Quality Engineer'}}

In [14]:
####### SANKEY #######

In [15]:
# Restructure with needed info for Sankey
Sankey_df = fulldf.groupby(["EmpNum","Topic","Status"])["Hours"].agg(['count']).reset_index()
Sankey_df

,EmpNum,Topic,Status,count
0,29436,CAPA,Late,2
1,29436,CAPA,On Time,4
2,29436,Inspection,Late,1
3,29436,Inspection,On Time,4
4,29436,New Product,Late,1
5,29436,New Product,On Time,4
6,29436,Training,Late,1
7,29436,Training,On Time,1
8,32301,Audit,Late,3
9,32301,Audit,On Time,9


In [16]:
Sankey_df["count"].sum() 

169

In [17]:
ontimeSankey = []
for topic in TopicList:
    
    for status in ["On Time", "Late"]:
        numdict = Sankey_df[(Sankey_df["Topic"]==topic)&
                             (Sankey_df["Status"]==status)][["EmpNum","count"]].set_index("EmpNum").to_dict()
        numdict = numdict["count"]
        namedict = {}
        for num in list(infodict.keys()):
            try:
                namedict[num+"-"+infodict[num]["EmpName"]] = numdict[num]
            except: 
                namedict[num+"-"+infodict[num]["EmpName"]] = 0
        ontimeSankey.append({
        "sex": status,
        "ses": topic,
        "endpoint": namedict
                            })

In [18]:
ontimeSankey

[{'sex': 'On Time',
  'ses': 'CAPA',
  'endpoint': {'32301-J Scott': 2,
   '49570-Z Taylor': 4,
   '77030-B Cranston': 5,
   '77707-T Kwan': 0,
   '29436-K Hart': 4,
   '88927-T Oliver': 2}},
 {'sex': 'Late',
  'ses': 'CAPA',
  'endpoint': {'32301-J Scott': 3,
   '49570-Z Taylor': 3,
   '77030-B Cranston': 1,
   '77707-T Kwan': 4,
   '29436-K Hart': 2,
   '88927-T Oliver': 3}},
 {'sex': 'On Time',
  'ses': 'Inspection',
  'endpoint': {'32301-J Scott': 3,
   '49570-Z Taylor': 0,
   '77030-B Cranston': 1,
   '77707-T Kwan': 3,
   '29436-K Hart': 4,
   '88927-T Oliver': 2}},
 {'sex': 'Late',
  'ses': 'Inspection',
  'endpoint': {'32301-J Scott': 6,
   '49570-Z Taylor': 3,
   '77030-B Cranston': 3,
   '77707-T Kwan': 6,
   '29436-K Hart': 1,
   '88927-T Oliver': 3}},
 {'sex': 'On Time',
  'ses': 'New Product',
  'endpoint': {'32301-J Scott': 5,
   '49570-Z Taylor': 4,
   '77030-B Cranston': 5,
   '77707-T Kwan': 3,
   '29436-K Hart': 4,
   '88927-T Oliver': 1}},
 {'sex': 'Late',
  'ses': '

In [19]:
## Radar data##

In [20]:
people_df= dfDict["EmpAffinity"][["EmpNum","Title","Topic","SkillRating_1to5"]].astype({"EmpNum":"str"})
# people_df

In [21]:
# set up affinity and hours spent on topic with structure needed for radar chart
radardict = {}
for person in people_df["EmpNum"].unique():
    affinity = people_df[people_df["EmpNum"]==person][["Topic","SkillRating_1to5"]].set_index("Topic").to_dict()["SkillRating_1to5"]
    TopicHours = fulldf[fulldf["EmpNum"]==person][["Topic","Hours"]].groupby("Topic").sum("Hours").to_dict()["Hours"]
    radardict[person]={}
    radardict[person]["affinity"] = affinity
    radardict[person]["topicHours"]=TopicHours
    radardict[person]["name"] = infodict[person]["EmpName"]
    radardict[person]["title"] = infodict[person]["Title"]

In [33]:
# set up affinity and hours spent on topic with structure needed for radar chart
radardict = {}
for person in people_df["EmpNum"].unique():
    affinity = people_df[people_df["EmpNum"]==person][["Topic","SkillRating_1to5"]].set_index("Topic").to_dict()["SkillRating_1to5"]
    TopicHours = fulldf[fulldf["EmpNum"]==person][["Topic","Hours"]].groupby("Topic").sum("Hours").rank().to_dict()["Hours"]
    hoursDict = {}
    for topic in TopicList:
        if topic in list(TopicHours.keys()):
            hoursDict[topic] = TopicHours[topic]
        else:
            hoursDict[topic] = 0        
    radardict[person]={}
    radardict[person]["affinity"] = affinity
    radardict[person]["topicHours"]=hoursDict
    radardict[person]["name"] = infodict[person]["EmpName"]
    radardict[person]["title"] = infodict[person]["Title"]

In [34]:
radardict

{'29436': {'affinity': {'Training': 2,
   'CAPA': 2,
   'Inspection': 4,
   'New Product': 1,
   'Audit': 3},
  'topicHours': {'CAPA': 4.0,
   'Inspection': 2.0,
   'New Product': 3.0,
   'Audit': 0,
   'Training': 1.0},
  'name': 'K Hart',
  'title': 'Inspector'},
 '32301': {'affinity': {'CAPA': 5,
   'Inspection': 4,
   'New Product': 4,
   'Audit': 4,
   'Training': 5},
  'topicHours': {'CAPA': 3.0,
   'Inspection': 2.0,
   'New Product': 4.0,
   'Audit': 5.0,
   'Training': 1.0},
  'name': 'J Scott',
  'title': 'Quality Supervisor'},
 '49570': {'affinity': {'Inspection': 4,
   'New Product': 1,
   'Audit': 4,
   'Training': 2,
   'CAPA': 3},
  'topicHours': {'CAPA': 5.0,
   'Inspection': 2.0,
   'New Product': 4.0,
   'Audit': 3.0,
   'Training': 1.0},
  'name': 'Z Taylor',
  'title': 'Inspector'},
 '77030': {'affinity': {'New Product': 2,
   'Audit': 5,
   'Training': 4,
   'CAPA': 4,
   'Inspection': 2},
  'topicHours': {'CAPA': 3.0,
   'Inspection': 1.0,
   'New Product': 4.0,
 

In [25]:
#### Add in Goal Levels

In [47]:
allhoursdf = fulldf.groupby("Topic").sum("Hours").reset_index()
allhoursdf["%"] = allhoursdf['Hours'] / allhoursdf["Hours"].sum()

In [51]:
hrsxferdict = allhoursdf[["Topic","%"]].set_index("Topic").to_dict()["%"]

In [52]:
topicGoals = dfDict["TopicGoals"].set_index("Topic").to_dict(orient="index")
for topic in list(topicGoals.keys()):
    topicGoals[topic]["Actual"] = hrsxferdict[topic]

In [53]:
topicGoals

{'CAPA': {'Goal': 0.2, 'std_hours': 8, 'Actual': 0.2820355610055181},
 'Inspection': {'Goal': 0.25, 'std_hours': 10, 'Actual': 0.12630288166768852},
 'New Product': {'Goal': 0.3, 'std_hours': 12, 'Actual': 0.23911710606989578},
 'Audit': {'Goal': 0.15, 'std_hours': 6, 'Actual': 0.2832618025751073},
 'Training': {'Goal': 0.1, 'std_hours': 4, 'Actual': 0.06928264868179031}}

In [28]:
### Create json structure

In [54]:
outdata = {
    "Goals": topicGoals,
    "radarData": radardict,
    "sankeyData": ontimeSankey
}

In [30]:
outdata

{'Goals': {'CAPA': {'Goal': 0.2, 'std_hours': 8},
  'Inspection': {'Goal': 0.25, 'std_hours': 10},
  'New Product': {'Goal': 0.3, 'std_hours': 12},
  'Audit': {'Goal': 0.15, 'std_hours': 6},
  'Training': {'Goal': 0.1, 'std_hours': 4}},
 'radarData': {'29436': {'affinity': {'Training': 2,
    'CAPA': 2,
    'Inspection': 4,
    'New Product': 1,
    'Audit': 3},
   'topicHours': {'CAPA': 4.0,
    'Inspection': 2.0,
    'New Product': 3.0,
    'Training': 1.0},
   'name': 'K Hart',
   'title': 'Inspector'},
  '32301': {'affinity': {'CAPA': 5,
    'Inspection': 4,
    'New Product': 4,
    'Audit': 4,
    'Training': 5},
   'topicHours': {'Audit': 5.0,
    'CAPA': 3.0,
    'Inspection': 2.0,
    'New Product': 4.0,
    'Training': 1.0},
   'name': 'J Scott',
   'title': 'Quality Supervisor'},
  '49570': {'affinity': {'Inspection': 4,
    'New Product': 1,
    'Audit': 4,
    'Training': 2,
    'CAPA': 3},
   'topicHours': {'Audit': 3.0,
    'CAPA': 5.0,
    'Inspection': 2.0,
    'New Pr

In [55]:
json_string = json.dumps(outdata)
# print(json_string)
with open('json_data.json', 'w') as outfile:
    outfile.write(json_string)